In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats

import time

import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import StrMethodFormatter

## Single-layer Perceptron for Equivalence Logic

## Compare Transfer Functions

## Learning Rate

## Double-layer Perceptron for XOR Logic